In [4]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;


//If all entries of x are One, the Oracle returns |y> = |1>
operation andOracle (x : Qubit[], y : Qubit) : Unit {
    body {
        (Controlled X)(x, y);
    }
    adjoint auto;
}

//If all entries of x are Zero, the Oracle returns |y> = |0>
operation orOracle (x : Qubit[], y : Qubit) : Unit {
    body {
        ApplyToEachA(X, x);
        Controlled X (x, y);
        X(y);
        ApplyToEachA(X, x);
    }
    adjoint auto;
}

//Given two qubits, XOR gate if 01 or 10 |y> = |1>
operation xorOracle2 (x : Qubit[], y : Qubit) : Unit {
    X(y);
    Controlled X(x, y);
    X(x[0]);
    X(x[1]);
    Controlled X(x, y);
    X(x[0]);
    X(x[1]);
}

//Puts a state |000...0> into the GHZ state |000..0> + |111..1>/sqrt2
operation buildGHZ (qs : Qubit[]) : Unit{
    body {
        H(qs[0]);
        for (i in 1..Length(qs)-1){
            CNOT(qs[0], qs[i]);
        }
    }
    adjoint auto;
}

//Puts a state |000...0> into the GHZ state |100..0> + |010..0> + ... |000..1>/sqrtN
operation buildW (qs : Qubit[]) : Unit is Adj + Ctl {
    body {
        let N = Length(qs);
        Ry(2.0*ArcSin(Sqrt(1.0/(IntAsDouble(N)))), qs[0]);
        if (N > 1){
            ((ControlledOnInt)(0, buildW))([qs[0]], qs[1...]);
        }
    }
    adjoint auto;
}

operation isAllZeros (qs : Qubit[]) : Int {
    return MeasureInteger(LittleEndian(qs)) == 0 ? 0 | 1;
}

//Generate Bell State 0 : |00>, 1 : |01>, 2 : |10>, 3 |11>
operation buildBell (qs : Qubit[], index : Int) : Unit {
    H(qs[0]);
    CNOT(qs[0], qs[1]);
        
    if (index%2 == 1){
        Z(qs[1]);
    }
    if (index/2 == 1){
        X(qs[0]);
    }
}

//Distinguish Bell State 0 : |00>, 1 : |01>, 2 : |10>, 3 : |11>
operation distinguishBell (qs : Qubit[]) : Int {
    CNOT(qs[0], qs[1]);
    H(qs[0]);
    
    let m1 = M(qs[0]) == One ? 1 | 0;
    let m2 = M(qs[1]) == One ? 1 | 0;
    
    return 2*m2 + m1;
}

//Reverse the Qubit string
operation reverseState (qs : Qubit[]) : Unit {
    let N = Length(qs);
    for (i in 0..(N/2)-1){
        SWAP(qs[i], qs[N-i-1]);
    }
}

//Create a copy of a the given state
operation stateCopy (qs : Qubit[], copy : Qubit[]) : Unit {
    for (i in 0..Length(qs)-1){
        CNOT(qs[i], copy[i]);
    }
}

//takes a integer array of 0/1s and creates an equivalent qubit string 
operation arrToQbit (bits : Int[], qs : Qubit[]) : Unit {
    for (i in 0..Length(qs)-1){
        if (bits[i] == 1){
            X(qs[i]);
        }
    }
}

//Teleport 2 qubits + message
operation Teleport (qAlice : Qubit, qBob : Qubit, qMessage : Qubit) : Unit {
    H(qAlice);
    CNOT(qAlice, qBob);
    CNOT(qMessage, qAlice);
    H(qMessage);
    
    (Controlled Z)([qMessage], qBob);
    (Controlled X)([qAlice], qBob);
}

//Superdensecoding protocol
operation SuperdenseCodingProtocol (b1 : Bool, b2 : Bool) : (Bool, Bool) {
    using ((qAlice, qBob) = (Qubit(), Qubit())){
        H(qAlice);
        CNOT(qAlice, qBob);
        
        //MCMAHON BOOK P236
        if (b1){
            Z(qAlice);
        }
        if (b2){
            X(qAlice);
        }
        
        CNOT(qAlice, qBob);
        H(qAlice);
        
        //Measure the qubit at Z basis, and reset it
        return ((MResetZ(qAlice) == One, MResetZ(qBob) == One));
    }
}

/snippet_.qs(1,90): warning QS6003: The namespace is already open.
/snippet_.qs(2,6): warning QS6003: The namespace is already open.
/snippet_.qs(13,9): warning QS3003: Deprecated syntax. An argument tuple is expected where "..." indicates the position of the declaration arguments, e.g. "(...)" for body and adjoint, "(cs, ...)" for controlled and controlled adjoint.
/snippet_.qs(54,9): warning QS3003: Deprecated syntax. An argument tuple is expected where "..." indicates the position of the declaration arguments, e.g. "(...)" for body and adjoint, "(cs, ...)" for controlled and controlled adjoint.
/snippet_.qs(21,9): warning QS3003: Deprecated syntax. An argument tuple is expected where "..." indicates the position of the declaration arguments, e.g. "(...)" for body and adjoint, "(cs, ...)" for controlled and controlled adjoint.
/snippet_.qs(43,9): warning QS3003: Deprecated syntax. An argument tuple is expected where "..." indicates the position of the declaration arguments, e.g. "(..

andOracle, arrToQbit, buildBell, buildGHZ, buildW, distinguishBell, isAllZeros, orOracle, reverseState, stateCopy, SuperdenseCodingProtocol, Teleport, xorOracle2

In [38]:
//////////////A1
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;

operation Solve(unitary : (Qubit => Unit is Adj+Ctl)) : Int {
    mutable ans = 0;
    using (q  = Qubit()){
      unitary(q);
      set ans = M(q)==One ? 1 | 0;
      Reset(q);
    }
    return ans;
}

operation test() : Unit {
    let n = 0;
    using(qs = Qubit[n]){

      Message("Tested");
      ResetAll(qs);
    }
}


/snippet_.qs(2,8): warning QS6003: The namespace is already open.
/snippet_.qs(3,8): warning QS6003: The namespace is already open.


Solve, test

In [195]:
//////////////A2
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;

operation Solve() : Int {
    mutable ans = 0;
    using (q = Qubit()){
      H(q);
      X(q);
      Y(q);
      Y(q);
      //OPERATION
      X(q);
      Z(q);
      X(q);
      //OPERATION
      Y(q);
      Y(q);
      X(q);
      H(q);
      DumpMachine();
      Reset(q);
    }
    return ans;
}


/snippet_.qs(2,6): warning QS6003: The namespace is already open.
/snippet_.qs(3,6): warning QS6003: The namespace is already open.


Solve

In [196]:
%simulate Solve

|0⟩	0 + 0𝑖
|1⟩	1.0000000000000002 + 0𝑖

0

In [56]:
////////////////////////B1

/snippet_.qs(1,113): error QS1002: An opening bracket has not been closed.


In [ ]:
////////////////////////B2

In [70]:
////////////////////////C1
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;

operation Solve (qs : Qubit[]) : Unit {
    PrepareUniformSuperposition(3, LittleEndian(qs));
    ApplyToEachA(Y, qs);
    ApplyToEachA(Z, qs);
    //ApplyToEachA(X, qs);
}

operation testC() : Unit {
    let n = 2;
    using(qs = Qubit[n]){
      Solve(qs);
      DumpMachine();
      Message("Tested");
      ResetAll(qs);
    }
}

/snippet_.qs(2,6): warning QS6003: The namespace is already open.
/snippet_.qs(3,6): warning QS6003: The namespace is already open.


Solve, testC

In [65]:
//////////////A3
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;

operation Solve(unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    mutable ans = 0;
    using (q  = Qubit()){
      H(q);
      unitary(q);
      unitary(q);
      H(q);
      set ans = M(q) == One ? 1 | 0;
      Reset(q);
    }
    return ans;
}

operation test() : Unit {
    let n = 1;
    using(qs = Qubit()){
      Solve(Z);
      Message("Tested");
      ResetAll(qs);
    }
}

/snippet_.qs(26,7): error QS5011: Expecting expression of type Unit. Only expressions of type Unit can be used as expression statements.
/snippet_.qs(28,15): error QS6210: The type of the given argument does not match the expected type. Got an argument of type Qubit, expecting one of type Qubit[] instead.


In [66]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;

operation Solve(unitary : (Qubit[] => Unit is Adj+Ctl)) : Int {
    mutable ans = 1;
    using (q  = Qubit[2]){
        unitary(q);
        if (M(q[1]) == One){
            set ans = 0;
        }
        ResetAll(q);
        }
    return ans;
}

/snippet_.qs(1,90): warning QS6003: The namespace is already open.
/snippet_.qs(2,6): warning QS6003: The namespace is already open.


Solve

In [71]:
%simulate testC

|0⟩	0 + 0𝑖
|1⟩	0.5773502691896263 + -3.5352507957496944E-17𝑖
|2⟩	0.5773502691896263 + -3.5352507957496944E-17𝑖
|3⟩	0.5773502691896263 + -3.535250795749694E-17𝑖

Tested


()

In [357]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;

operation buildBell (qs : Qubit[], index : Int) : Unit {
    H(qs[0]);
    CNOT(qs[0], qs[1]);
        
    if (index%2 == 1){
        Z(qs[1]);
    }
    if (index/2 == 1){
        X(qs[0]);
    }
}

operation Solve (qs : Qubit[]) : Unit {
    using (q = Qubit()){
        H(q);
        buildBell(qs, 1);
        let m = M(qs[1]);
        DumpMachine();
        Reset(q);
    }
    //let m = M(qs[0]);
}

operation main () : Unit {
    using (qs = Qubit[2]){
        Solve(qs);
        //DumpMachine();
        ResetAll(qs);
    }
}

/snippet_.qs(1,90): warning QS6003: The namespace is already open.
/snippet_.qs(2,6): warning QS6003: The namespace is already open.


buildBell, main, Solve

In [358]:
%simulate main

|0⟩	0.7071067811865475 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0.7071067811865475 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖

()